In [1]:
import sys
sys.path.append("/Users/lochuynhquang/Documents/GPG_3D_yardplan/yardplan_creator")
from gpg import *


depot_name = "Song Than depot"
outfile = "./tbd.json"

In [2]:
tbd_layout = Layout("./tbd.svg")
# tbd_layout.scale(cox=3, coy=3)
# tbd_layout.translate(-400, -1700)

tbd_layout.shape[1].visible = 0.2
tbd_layout.shape[2].visible = 0.2
tbd_layout.shape[9].visible = 0.2
tbd_layout.shape[10].visible = 0


#button
tbd_layout.shape[14].visible = 0
tbd_layout.shape[15].visible = 0
tbd_layout.shape[16].visible = 0
tbd_layout.shape[17].visible = 0
tbd_layout.shape[18].visible = 0
tbd_layout.shape[19].visible = 0
tbd_layout.shape[20].visible = 0
tbd_layout.shape[21].visible = 0.2
tbd_layout.shape[22].visible = 0.2

btn0 = Button(tbd_layout.shape[14].seq[3], "Area 1a", angle=-1.1257373675363425);
btn1 = Button(tbd_layout.shape[15].seq[3], "Area 1b", angle=-1.0222393428930787);
btn2 = Button(tbd_layout.shape[16].seq[3], "Area 1c", angle=0.18448130193580065);
btn3 = Button(tbd_layout.shape[17].seq[3], "Area 1d", angle=-0.1095892237327239 + math.pi/2);
btn4 = Button(tbd_layout.shape[18].seq[1], "Area 2a", angle=0.05462880558742251);
btn6 = Button(tbd_layout.shape[20].seq[1], "Area 2c", angle=-1.4809118703171884);

ground0 = Ground(1,4, offsetX=tbd_layout.shape[1].seq[3].x, offsetY=tbd_layout.shape[1].seq[3].y, offsetZ=0, angle=0.445058959258554, button=btn0, height=980, width=1060);
ground1 = Ground(1,3, offsetX=tbd_layout.shape[1].seq[3].x-35, offsetY=tbd_layout.shape[1].seq[3].y+25, offsetZ=0, angle=0.5485569839018177, height=1020, width=1060, button=btn1)
ground2 = Ground(1,0, offsetX=tbd_layout.shape[1].seq[3].x, offsetY=tbd_layout.shape[1].seq[3].y, height=840, width=1080, offsetZ=0, angle=0.18448130193580065, button=btn2)
ground3 = Ground(1,0, offsetX=tbd_layout.shape[1].seq[1].x+20, offsetY=tbd_layout.shape[1].seq[1].y-135, height=1040, width=1140, offsetZ=0, angle=-0.1095892237327239, button=btn3)
ground4 = Ground(2,0, offsetX=tbd_layout.shape[2].seq[0].x, offsetY=tbd_layout.shape[2].seq[0].y, height=300, width=700, offsetZ=0, angle=0.05462880558742251, button=btn4)
ground6 = Ground(2,0, offsetX=tbd_layout.shape[2].seq[0].x, offsetY=tbd_layout.shape[2].seq[0].y, height=320, width=700, offsetZ=0, angle=0.08988445647770797, button=btn6)

# add warehouse here
h1 = House(0, "house 1", tbd_layout.shape[3],0,2,200,0.05235987755982988)
h2 = House(0, "house 2", tbd_layout.shape[4],0,2,200,0.05235987755982988)
h3 = House(0, "house 3", tbd_layout.shape[5],0,2,200,0.05235987755982988)
h4 = House(0, "house 4", tbd_layout.shape[6],0,2,200,0.05235987755982988)
h5 = House(0, "house 5", tbd_layout.shape[7],0,2,200,0.05235987755982988)
h6 = House(0, "house 6", tbd_layout.shape[8],0,2,200,0.05235987755982988)
# h7 = House(0, "house 7", tbd_layout.shape[9],0,2,200,0.05235987755982988)
# h8 = House(0, "house 8", tbd_layout.shape[10],0,2,200,0.05235987755982988)
h9 = House(0, "house 9", tbd_layout.shape[11],0,2,200,0.05235987755982988)
h10 = House(0, "house 10", tbd_layout.shape[12],0,2,200,0.05235987755982988)
h11 = House(0, "house 11", tbd_layout.shape[13],0,2,200,0.05235987755982988)
# ruaCont = House(2, "Rua container", tbd_layout.shape[4],3,1,20, offsetX=34, offsetY=-74)
tbd = Depot("Tây Nam Bình Dương Depot", tbd_layout, ground=[ground0, ground1, ground2, ground3, ground4, ground6], house=[h1,h2,h3,h4,h5,h6,h9,h10,h11], container_scale=1.786, scale=0.4, defaultGround=2)
# etd = Depot("etd", tbd_layout, ground=[ground1, ground2], offset=Point(-450,300), house=[wh1, wh2, wh3, ruaCont],container_scale=4.25, scale=0.4)

In [3]:
jn = (depotEncoder().encode(tbd))
ofile = open(outfile, "w")
ofile.write(jn)
ofile.close()